In [1]:
import pandas as pd
import numpy as np
import os

# --- Helper Functions for File I/O ---

def _read_file(file_path: str) -> pd.DataFrame | None:
    """Reads a DataFrame from a CSV or XLSX file."""
    extension = os.path.splitext(file_path)[1].lower()
    
    try:
        if extension == '.csv':
            # Use read_csv for CSV files
            df = pd.read_csv(file_path, sep=',')
        elif extension in ('.xlsx', '.xls'):
            # Use read_excel for Excel files
            # You might need to specify the sheet name if it's not the first one
            df = pd.read_excel(file_path)
        else:
            print(f"Error: Unsupported file type: {extension}. Must be CSV or XLSX.")
            return None

        # Standard cleanup for all files
        df.columns = df.columns.str.strip()
        return df

    except FileNotFoundError:
        print(f"Error: Input file '{file_path}' not found.")
        return None
    except Exception as e:
        print(f"Error loading file '{file_path}': {e}")
        return None

def _write_file(df: pd.DataFrame, file_path: str):
    """Writes a DataFrame to a CSV or XLSX file."""
    extension = os.path.splitext(file_path)[1].lower()
    
    if extension == '.csv':
        df.to_csv(file_path, index=False, sep=',')
    elif extension in ('.xlsx', '.xls'):
        # For Excel, we use a single sheet named 'FamilyData'
        df.to_excel(file_path, index=False, sheet_name='FamilyData')
    else:
        raise ValueError(f"Unsupported output file type: {extension}")


# --------------------------------------------------------------------------------

def fill_family_relationships(input_file: str, output_file: str) -> str | None:
    """
    Reads family data, calculates ChildID and SiblingID based on parent columns,
    fills in missing values in the existing 'ChildID' and 'SiblingID' columns,
    and saves the resulting DataFrame.
    
    Args:
        input_file: The path to the input CSV or XLSX file.
        output_file: The path for the output CSV or XLSX file.
        
    Returns:
        The path to the output file if successful, otherwise None.
    """
    print(f"--- Step 1: Filling SiblingID and ChildID and saving to '{output_file}' ---")
    
    # Load the data using the helper function
    df = _read_file(input_file)
    if df is None:
        return None

    # Columns required for calculation (must be treated as numbers)
    id_cols_calc = ['PersonID', 'PartnerID', 'FatherID', 'MotherID']
    for col in id_cols_calc:
        # Coerce errors to NaN, then cast to Int64 (pandas integer type that supports NaN)
        df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')
        
    # Prepare ID columns for filling: ensure they exist and convert to string/object
    id_cols_fill = ['SiblingID', 'ChildID']
    for col in id_cols_fill:
        if col not in df.columns:
            df[col] = ''
        # Convert existing fill columns to string/object type, treating explicit empty strings as NaN for filling
        df[col] = df[col].astype(str).str.strip().replace({None: np.nan, 'nan': np.nan, '': np.nan})


    # --- 1. Calculate and Fill Child IDs ---
    children_map = {}
    for _, row in df.iterrows():
        child_id = row['PersonID']
        father_id = row['FatherID']
        mother_id = row['MotherID']

        if pd.notna(father_id):
            # Using int() here is safe because we already cast to Int64
            children_map.setdefault(int(father_id), []).append(child_id)
        
        if pd.notna(mother_id):
            children_map.setdefault(int(mother_id), []).append(child_id)

    # Convert calculated list of IDs to a comma-separated string series
    calculated_children_series = df['PersonID'].map(children_map).fillna('').apply(
        lambda x: ', '.join(map(str, x)) if isinstance(x, list) else ''
    )
    
    # Fill missing values in original 'ChildID' column with the calculated series
    df['ChildID'] = df['ChildID'].fillna(
        calculated_children_series.replace('', np.nan)
    ).fillna('')


    # --- 2. Calculate and Fill Sibling IDs (Full Siblings only) ---
    siblings_map = {}
    sibling_groups = df.dropna(subset=['FatherID', 'MotherID']).groupby(['FatherID', 'MotherID'])

    for _, group in sibling_groups:
        group_ids = group['PersonID'].tolist()
        
        if len(group_ids) > 1:
            for person_id in group_ids:
                # Exclude the person themselves from their sibling list
                siblings = [str(int(id)) for id in group_ids if id != person_id]
                # Using int() is safe because we already cast to Int64
                siblings_map[int(person_id)] = ', '.join(siblings)

    # Map the list of siblings back to the DataFrame
    calculated_siblings_series = df['PersonID'].map(siblings_map).fillna('')
    
    # Fill missing values in original 'SiblingID' column
    df['SiblingID'] = df['SiblingID'].fillna(
        calculated_siblings_series.replace('', np.nan)
    ).fillna('')

    # Save the intermediate result using the helper function
    try:
        _write_file(df, output_file)
        print(f"Successfully created intermediate file '{output_file}'.")
        return output_file
    except Exception as e:
        print(f"Error saving file '{output_file}': {e}")
        return None

# --------------------------------------------------------------------------------

def calculate_generations(input_file: str, output_file: str):
    """
    Reads the data (with filled SiblingID/ChildID), calculates the Generation 
    number for each person based on all related people with known generations, 
    checks for consistency, and saves/prints the final result.
    
    Args:
        input_file: The path to the input CSV or XLSX file (must have filled ID columns).
        output_file: The path for the output CSV or XLSX file.
    """
    print(f"\n--- Step 2: Calculating Generation numbers and saving to '{output_file}' ---")
    
    # Load the data using the helper function
    df = _read_file(input_file)
    if df is None:
        return

    # --- Setup Data Types ---
    # Ensure ID columns are treated as integers
    id_cols = ['PersonID', 'PartnerID', 'FatherID', 'MotherID']
    for col in id_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')

    # Ensure Generation column exists and is numeric (Int64 allows NaN)
    if 'Generation' not in df.columns:
        df['Generation'] = np.nan
    df['Generation'] = pd.to_numeric(df['Generation'].astype(str).str.strip(), errors='coerce').astype('Int64')

    # Initialize Generation Map with existing known generations
    gen_map = df.set_index('PersonID')['Generation'].dropna().apply(int).to_dict()
    
    # Create a copy sorted by PersonID to ensure we rely on already processed (lower ID) people
    df_sorted = df.sort_values(by='PersonID').copy()

    # --- Iteration and Calculation ---
    
    for _, row in df_sorted.iterrows():
        person_id = row['PersonID']
        
        # Skip anyone who already has a generation defined
        if pd.notna(row['Generation']):
            continue

        calculated_gens = set()
        
        # 1. Check Parents: Child Gen = Parent Gen + 1
        for parent_id in [row['FatherID'], row['MotherID']]:
            if pd.notna(parent_id) and int(parent_id) in gen_map:
                calculated_gens.add(gen_map[int(parent_id)] + 1)
                
        # 2. Check Siblings: Sibling Gen = Sibling Gen (Same)
        # We need to look up the SiblingID from the *original* df as it was modified in step 1
        sibling_ids_str = str(df.loc[df['PersonID'] == person_id, 'SiblingID'].iloc[0])
        # Add cleanup step to remove artifacts like brackets before splitting
        sibling_ids_str = sibling_ids_str.replace('[', '').replace(']', '').replace("'", '').replace('"', '')

        if sibling_ids_str and sibling_ids_str != 'nan':
            # Safely split and convert to int
            for sibling_id in [int(x.strip()) for x in sibling_ids_str.split(',') if x.strip()]:
                if sibling_id in gen_map:
                    calculated_gens.add(gen_map[sibling_id])
                    
        # 3. Check Children: Parent Gen = Child Gen - 1
        # We need to look up the ChildID from the *original* df as it was modified in step 1
        child_ids_str = str(df.loc[df['PersonID'] == person_id, 'ChildID'].iloc[0])
        # Add cleanup step to remove artifacts like brackets before splitting
        child_ids_str = child_ids_str.replace('[', '').replace(']', '').replace("'", '').replace('"', '')

        if child_ids_str and child_ids_str != 'nan':
            for child_id in [int(x.strip()) for x in child_ids_str.split(',') if x.strip()]:
                if child_id in gen_map:
                    calculated_gens.add(gen_map[child_id] - 1)
            
        # 4. Check Partner: Partner Gen = Self Gen (Same)
        partner_id = row['PartnerID']
        if pd.notna(partner_id) and int(partner_id) in gen_map:
            calculated_gens.add(gen_map[int(partner_id)])

        
        # --- Analysis and Update ---
        
        if not calculated_gens:
            # Cannot determine generation yet
            continue
            
        # Check for conflicts (different calculated generations)
        if len(calculated_gens) > 1:
            print(f"--- CONFLICT DETECTED ---")
            print(f"PersonID: {person_id}")
            print(f"Conflicting calculated generations: {list(calculated_gens)}")
            return # Stop and return on first conflict

        # If no conflict, update the generation map and the main dataframe
        new_gen = list(calculated_gens)[0]
        gen_map[person_id] = new_gen
        df.loc[df['PersonID'] == person_id, 'Generation'] = new_gen
        
    # --- Final Output and Saving ---

    # Print the final DataFrame in a nicely formatted, readable way using df.to_string()
    print("\n--- Final Filled Family Data with Generations (Readably Formatted) ---")
    print(df.to_string())
    print("----------------------------------------------------------------------\n")
    
    # Save the final DataFrame using the helper function
    try:
        _write_file(df, output_file)
        print(f"Successfully created '{output_file}' with filled relationship and generation data.")
    except Exception as e:
        print(f"Error saving file '{output_file}': {e}")


if __name__ == '__main__':
    # Define file extension variable to easily switch between '.csv' and '.xlsx'
    FILE_EXT = '.xlsx' # <-- Change this to .csv or '.xlsx' to process Excel files
    
    # Define file names
    INPUT_FILE = f'family_data{FILE_EXT}'
    INTERMEDIATE_FILE = f'family_data_filled{FILE_EXT}'
    OUTPUT_FILE = f'family_data_filled_gen{FILE_EXT}'
    
    # Step 1: Fill SiblingID and ChildID
    filled_file = fill_family_relationships(INPUT_FILE, INTERMEDIATE_FILE)
    
    # Step 2: Calculate Generations using the result from Step 1
    if filled_file:
        calculate_generations(filled_file, OUTPUT_FILE)

--- Step 1: Filling SiblingID and ChildID and saving to 'family_data_filled.xlsx' ---
Successfully created intermediate file 'family_data_filled.xlsx'.

--- Step 2: Calculating Generation numbers and saving to 'family_data_filled_gen.xlsx' ---

--- Final Filled Family Data with Generations (Readably Formatted) ---
    PersonID          Name       Name-ru LastName-ru Gender   BirthDate  DeathDate  PartnerID  FatherID  MotherID   SiblingID         ChildID  Generation                       PlaceBirth                       PlaceNow                                                         Occupation             Photo
0          1        Leonid        Леонид    Савченко      M  12.01.1945        NaN          2        27        28          21         3, 4, 5           4  Palatka, Magadan region, Russia  Sokol, Magadan region, Russia                                               Science, engineering    id1-leonid.png
1          2         Elena         Елена    Савченко      F  12.01.1946       